# K-Nearest Neighbors Classifier using Custom Algorithm

In [1]:
from keras.datasets import mnist 
from sklearn.metrics import confusion_matrix as cm
from sklearn.utils import shuffle
from sklearn.metrics import classification_report as cr
import numpy as np
import random
import pandas as pd
import operator as op
import timeit as tt
from operator import itemgetter as ig

Custom method to calculate Euclidean Distance

In [2]:
def eucDist(x1, x2): 
    return np.sqrt(np.sum(x1-x2)**2)

Defining the kNN Algorithm

In [3]:
# Source: https://medium.com/analytics-vidhya/a-beginners-guide-to-knn-and-mnist-handwritten-digits-recognition-using-knn-from-scratch-df6fb982748a
class kNN:
    def __init__(self, n_neighbors):
        self.K = n_neighbors
    
    def fit(self, train_X, train_y):
        self.train_X = train_X
        self.train_y = train_y
    
    def predict(self, test_X):
        predictions = [] 
        for i in range(len(test_X)):
            dist = np.array([eucDist(test_X[i], t_X) for t_X in self.train_X])
            distSort = dist.argsort()[:self.K]
            nCount = {} # Count of neighbors
            for idx in distSort:
                if self.train_y[idx] in nCount:
                    nCount[self.train_y[idx]] += 1
                else:
                    nCount[self.train_y[idx]] = 1
            sorted_nCount = sorted(nCount.items(), key=ig(1), reverse=True)
            predictions.append(sorted_nCount[0][0]) 
        return predictions

Calculations and definitions of variables

In [4]:
k = 44 #Since our USNs end with 17 and 44, the 'k' value is 44
dp = 7100 #number of data points; (17 + 44) x 100 + 1000 = 7100
trp = 5680 #training points; 80% of 7100 = 5680
tp = 1420 #testing points; 20% of 7100 = 1420
n = 10 # Will be used for matrices and for-loops

Defining training and testing datas

In [5]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = train_X[:dp] # Resizing train_X with our required data points
train_X = np.reshape(train_X, (dp, 28*28)) 
train_y = train_y[:dp] # Resizing train_y with our required data points
test_X = test_X[:dp] # Resizing test_X with our required data points
test_X = np.reshape(test_X, (dp, 28*28))
test_y = test_y[:dp] # Resizing test_X with our required data points

Custom shuffle method for training data

In [14]:
# implementing mnist from keras in sklearn's train_test_split was complicated
def shufTrainData():
    X_shuf, y_shuf = shuffle(train_X, train_y, random_state = None)
    return (X_shuf, y_shuf)

Custom shuffle method for testing data

In [7]:
def shufTestData():
    X_shuf, y_shuf = shuffle(test_X, test_y, random_state = None)
    return (X_shuf, y_shuf)

Method to calculate the average runtime

In [8]:
def calcAvg(tAvg, total_1):
    if i == 0:
        tAvg = total_1
        return tAvg
    elif i != 0:
        tAvg = (tAvg + total_1)/(2)
        return tAvg

Cross Validation

In [9]:
cm_0 = np.zeros((n, n)) # Creating a temporary null matrix
# To calculate the precision of each digit
# Source: Sai Nishwanth, USN: ENG21AM3031
prec = 0

# To calculate total runtime for k iterations 
# Source: https://stackoverflow.com/questions/5622976/how-do-you-calculate-program-run-time-in-python
start_0 = tt.default_timer()
tAvg = 0
knn = kNN(n_neighbors = k)

for i in range(k): 
    start_1 = tt.default_timer() # To calculate runtime for each iteration
    print(f'Starting iteration - {i}') # To keep a track of how many iterations have finished
    X_train, y_train = shufTrainData()
    X_test, y_test = shufTestData()
    X_train = X_train[:trp] # Training with 80% of data
    y_train = y_train[:trp] 
    knn.fit(X_train, y_train)
    X_test = X_test[:tp] # Testing with 20% of data
    y_test = y_test[:tp]
    y_pred = knn.predict(X_test)
    cm_2 = cm(y_test, y_pred)
    cm_1 = cm_0 + cm_2
    cm_0 = cm_1
    prec += cm_0.diagonal()/cm_1.sum(axis = 0)
    print(f'Ending iteration - {i}')
    stop_1 = tt.default_timer() # To calculate runtime for each iteration
    total_1 = (stop_1 - start_1)
    tAvg = calcAvg(tAvg, total_1)
    print(f'Total runtime for iteration \'{i}\': {total_1} seconds.\n')
    
# To calculate total runtime for k iterations
stop_0 = tt.default_timer()
total_0 = (stop_0 - start_0)/60 # Since time is stored in seconds and it is easier to read in minutes
for i in range(n):
    print(f'Precision of number {i} - {(prec[i]/k)*100} %')
    
cmAvg = cm_1/k 
print(cmAvg)
print(f'Total runtime for {k} iterations: {total_0} minutes.') 
print(f'Average runtime per iterations: {tAvg} seconds.') 

Starting iteration - 0
Ending iteration - 0
Total runtime for iteration '0': 37.42399975 seconds.

Starting iteration - 1
Ending iteration - 1
Total runtime for iteration '1': 37.5163645 seconds.

Starting iteration - 2
Ending iteration - 2
Total runtime for iteration '2': 37.795837417000016 seconds.

Starting iteration - 3
Ending iteration - 3
Total runtime for iteration '3': 37.751705125 seconds.

Starting iteration - 4
Ending iteration - 4
Total runtime for iteration '4': 38.000387875 seconds.

Starting iteration - 5
Ending iteration - 5
Total runtime for iteration '5': 37.811503875 seconds.

Starting iteration - 6
Ending iteration - 6
Total runtime for iteration '6': 37.41682720899999 seconds.

Starting iteration - 7
Ending iteration - 7
Total runtime for iteration '7': 37.45696004200005 seconds.

Starting iteration - 8
Ending iteration - 8
Total runtime for iteration '8': 37.78718183299998 seconds.

Starting iteration - 9
Ending iteration - 9
Total runtime for iteration '9': 37.54

Creating a Pandas Data Frame for better viewing of the result

In [11]:
df = pd.DataFrame(cmAvg)
df

,0,1,2,3,4,5,6,7,8,9
0,124.522727,7.386364,0.000000,0.000000,0.159091,0.000000,3.795455,0.204545,1.136364,0.045455
1,0.000000,161.795455,0.113636,0.431818,0.000000,0.000000,0.181818,0.000000,0.000000,0.000000
2,3.909091,58.045455,76.863636,1.159091,0.454545,0.000000,2.795455,2.931818,3.045455,0.204545
3,1.113636,51.113636,0.340909,81.954545,0.000000,0.431818,1.295455,2.818182,3.159091,2.545455
4,0.318182,29.250000,0.136364,0.000000,74.022727,0.000000,4.181818,0.409091,0.136364,32.477273
5,7.090909,33.977273,0.181818,8.113636,0.181818,49.068182,7.272727,2.318182,9.113636,9.977273
6,2.886364,16.772727,0.000000,0.000000,0.477273,0.000000,110.000000,0.181818,0.000000,0.000000
7,0.340909,30.500000,0.000000,0.000000,0.000000,0.000000,0.068182,102.022727,1.022727,8.909091
8,3.863636,57.295455,0.159091,1.863636,0.318182,0.477273,3.522727,3.340909,62.954545,5.681818
9,2.636364,19.795455,0.068182,0.522727,0.386364,0.022727,0.522727,3.090909,1.886364,116.227273


To calculate and verify the sum of all the elements in the matrix

In [13]:
s = 0
for i in range(n):
    for j in range(n):
        s += cmAvg[i][j]
        
s

1420.0